In [1]:
import serial

In [2]:
# Custom Exceptions
class Error(Exception):
    """Base class for exceptions in this module.
    :param msg: Error message associated with the exception
    :type msg: str
    :ivar msg: Error message associated with the exception
    """

    def __init__(self, msg):
        self.msg = msg

    def __str__(self):
        return self.msg


class CommandError(Error):
    """Exception raised sending a command to the SAML11."""

    pass


class ConnectError(Error):
    """Exception raised connecting to the SAML11."""

    pass

In [3]:
# https://github.com/nkinder/smart-card-removinator/blob/master/client/removinator/removinator.py
class SAML11:
    """Communicates to the SAML11 on the specified serial port.
    """

    def __init__(
        self,
        port="COM3",
        baudrate=9600,
        dsrdtr=True,
        bytesize=serial.EIGHTBITS,
        parity=serial.PARITY_EVEN,
        stopbits=serial.STOPBITS_TWO,
        DTR=True,
    ):
        """Opens a connection to the SAML11 on the specified serial port.
        """

        self.port = port
        self.baudrate = baudrate
        self.dsrdtr = dsrdtr
        self.bytesize = bytesize
        self.parity = parity
        self.stopbits = stopbits
        self.DTR = DTR

        self.last_result = ""
        self.last_response = ""

        # Open a connection to the SAML11.
        try:
            self.connection = serial.Serial(
                port=port,
                baudrate=baudrate,
                dsrdtr=dsrdtr,
                bytesize=bytesize,
                parity=parity,
                stopbits=stopbits,
            )
            self.connection.setDTR(DTR)
        except serial.SerialException as e:
            raise ConnectError(
                "Unable to open connection to SAML11 "
                "controller on port {0}".format(port)
            )

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.connection.close()

    def get_status(self):
        """
        Get the status of the SAML11
        :raises: :exc:`CommandError`
        """

        self.send_command("STA")
        # return json.loads(self.last_response.rstrip())
        return self.last_response

    def send_command(self, command):
        """
        Send a command to the SAML11
        Sends the specified command to the SAML11.  The command
        result (*OK* or *ERR_\\**) will be available in the *last_result*
        attribute of your *SAML11* object.  Similarly, the full command
        response will be available in the *last_response* attribute.
        If an error result is encountered, a CommandError will be
        raised.
        :param command: the command to send
        :type command: str
        :raises: :exc:`CommandError`
        """

        # Clear out the previous command result and response.
        self.last_result = ""
        self.last_response = ""

        # Send the command in a proper envelope to the SAML11.
        try:
            self.connection.write("#{0}\r".format(command).encode("utf-8"))
        except serial.SerialException as e:
            raise CommandError(
                "Serial error encountered sending command "
                '"{0}" ({1})'.format(command, e)
            )

        while 1:
            resp_line = self.connection.readline().decode("utf-8")
            # print(resp_line)
            if resp_line.startswith("OK"):
                self.last_result = resp_line.rstrip()
                break
            elif resp_line.startswith("ERR_"):
                self.last_result = resp_line.rstrip()
                raise CommandError(
                    '{0} encountered sending command "{1}"'.format(
                        self.last_result, command
                    )
                )

            # Concatenate multi-line responses.
            self.last_response += resp_line
    
    def set_debug(self, debug):
        """
        Enable or disable SAML11 debug output
        When enabled, the SAML11 will return verbose responses
        via serial when commands are executed.  The debug output from the
        previously run command will be available along with the standard
        command output in the *last_response* attribute of your
        *SAML11* object.
        :param debug: enable/disable debugging
        :type debug: bool
        :raises: :exc:`CommandError`
        """
        # Send the debug command.
        self.send_command('DBG')

        # Check the response to see if we are in the requested state.
        if (debug and (self.last_response.endswith('DBG_OFF'))):
            self.send_command('DBG')
        elif (not debug and (self.last_response.endswith('DBG_ON'))):
            self.send_command('DBG')

In [12]:
with SAML11() as board:
    print(board.get_status())
    board.send_command("?")
    print("Last result: {}".format(board.last_result))
    print("Last response:\n{}".format(board.last_response).replace("\n","\n\t"))
    board.send_command("STA")
    print("Last result: {}".format(board.last_result))
    print("Last response:\n{}".format(board.last_response).replace("\n","\n\t"))
    board.send_command("DBG")
    print("Last result: {}".format(board.last_result))
    print("Last response:\n{}".format(board.last_response).replace("\n","\n\t"))
    board.send_command("DBG")
    print("Last result: {}".format(board.last_result))
    print("Last response:\n{}".format(board.last_response).replace("\n","\n\t"))

Hi
Hello

Last result: OK
Last response:
	Hi
	--- SAML11 ---
	
	Command Format:
		Prefix:		#
		Terminator:	\r
	Commands:
		STA	Return some status
		DBG	Toggle debugging output	?	Show usage information
	
Last result: OK
Last response:
	Hi
	Hello
	
Last result: OK
Last response:
	Hi
	DBG_ON
	
Last result: OK
Last response:
	Hi
	[DBG] Received command "DBG"OK
	DBG_OFF
	
